adding a RunManager class in this increment to manage multiple runs and making experimentation easier as well as increasing the readability of code in pracFMNIST3.ipynb<br>
the RunManager class will essentially wrap up all the tensorboard calls in one place

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [2]:
use_cuda = True
torch.cuda.is_available()

True

In [0]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run',params.keys())
        runs = []
        for  v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [0]:
class RunManager():
    def __init__(self):

        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self,run,network,loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run 
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images,labels = next(iter(self.loader))

        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
        
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images',grid)
        self.tb.add_graph(self.network,images)

    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss',loss,self.epoch_count)
        self.tb.add_scalar('Accuracy',accuracy,self.epoch_count)

        for name,param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f'{name}.grad',param.grad,self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration

        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)

        df = pd.DataFrame.from_dict(self.run_data,orient='columns')

        clear_output(wait=True)     #specific to ipynbs
        display(df)                 #specific to ipynbs

    def track_loss(self,loss):
        self.epoch_loss += loss.item()*self.loader.batch_size

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct += self._get_num_correct(preds,labels)

    @torch.no_grad()
    def _get_num_correct(self,pred,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def save(self,fileName):

        pd.DataFrame.from_dict(
            self.run_data,
            orient="columns",
        ).to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json','w',encoding = 'utf-8') as f:
            json.dump(self.run_data,f,ensure_ascii = False,indent=4)



In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,12,5)

        self.fc1 = nn.Linear(12*4*4,120)
        self.fc2 = nn.Linear(120,60)
        self.out = nn.Linear(60,10)

    def forward(self,t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)

        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t

In [0]:
train_set = torchvision.datasets.FashionMNIST(
    root = "./data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    )
)

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)

In [0]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [1000,2000],
    shuffle = [True,False]
)

class RunManager():
    def __init__(self):

        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self,run,network,loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run 
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images,labels = next(iter(self.loader))

        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
        
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images',grid)
        self.tb.add_graph(self.network,images)

    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss',loss,self.epoch_count)
        self.tb.add_scalar('Accuracy',accuracy,self.epoch_count)

        for name,param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f'{name}.grad',param.grad,self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration

        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)

        df = pd.DataFrame.from_dict(self.run_data,orient='columns')

        clear_output(wait=True)     #specific to ipynbs
        display(df)                 #specific to ipynbs

    def track_loss(self,loss):
        self.epoch_loss += loss.item()*self.loader.batch_size

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct += self._get_num_correct(preds,labels)

    @torch.no_grad()
    def _get_num_correct(self,pred,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def save(self,fileName):

        pd.DataFrame.from_dict(
            self.run_data,
            orient="columns",
        ).to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json','w',encoding = 'utf-8') as f:
            json.dump(self.run_data,f,ensure_ascii = False,indent=4)



In [23]:
!rm -r runs

rm: cannot remove 'runs': No such file or directory


In [24]:
m = RunManager()
for run in RunBuilder.get_runs(params):
    
    network = Network()     
    if use_cuda and torch.cuda.is_available():
        network.cuda()

    loader = DataLoader(train_set,batch_size = run.batch_size,shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(),lr=run.lr)

    m.begin_run(run,network,loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:

            images = batch[0].cuda()
            labels = batch[1].cuda()

            preds = network(images)
            loss = F.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            m.track_loss(loss)
            m.track_num_correct(preds,labels)

        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,0.992876,0.620067,8.194866,8.831222,0.01,1000,True
1,1,2,0.497925,0.812383,8.287177,17.251443,0.01,1000,True
2,1,3,0.410849,0.849133,8.225430,25.611681,0.01,1000,True
3,1,4,0.374719,0.860783,8.340810,34.084576,0.01,1000,True
4,1,5,0.346941,0.872117,8.378311,42.604742,0.01,1000,True
5,2,1,1.055619,0.595517,8.239129,8.840413,0.01,1000,False
6,2,2,0.557011,0.782117,8.338158,17.327412,0.01,1000,False
7,2,3,0.466076,0.827883,8.390646,25.850688,0.01,1000,False
8,2,4,0.409344,0.849367,8.237872,34.223108,0.01,1000,False
9,2,5,0.368943,0.863483,8.216341,42.572786,0.01,1000,False


downloading runs for visualising on tensorboard

In [25]:
!zip -r runs3 runs

  adding: runs/ (stored 0%)
  adding: runs/Oct29_13-50-59_0eca79d7de00-Run(lr=0.01, batch_size=1000, shuffle=True)/ (stored 0%)
  adding: runs/Oct29_13-50-59_0eca79d7de00-Run(lr=0.01, batch_size=1000, shuffle=True)/events.out.tfevents.1572357059.0eca79d7de00.122.6 (deflated 51%)
  adding: runs/Oct29_13-53-07_0eca79d7de00-Run(lr=0.01, batch_size=2000, shuffle=False)/ (stored 0%)
  adding: runs/Oct29_13-53-07_0eca79d7de00-Run(lr=0.01, batch_size=2000, shuffle=False)/events.out.tfevents.1572357187.0eca79d7de00.122.9 (deflated 35%)
  adding: runs/Oct29_13-51-41_0eca79d7de00-Run(lr=0.01, batch_size=1000, shuffle=False)/ (stored 0%)
  adding: runs/Oct29_13-51-41_0eca79d7de00-Run(lr=0.01, batch_size=1000, shuffle=False)/events.out.tfevents.1572357101.0eca79d7de00.122.7 (deflated 51%)
  adding: runs/Oct29_13-52-24_0eca79d7de00-Run(lr=0.01, batch_size=2000, shuffle=True)/ (stored 0%)
  adding: runs/Oct29_13-52-24_0eca79d7de00-Run(lr=0.01, batch_size=2000, shuffle=True)/events.out.tfevents.15723